<img src='cuny.png' align='left'>
# DATA 602 Semester Project: Latent Semantic Analysis

Spring semester 2017  
By: Dmitriy Vecheruk  

  
- - -
  
## 0. Introduction

This interactive notebook is my submission for the CUNY MSDA DATA 602 Semester Project assignment. The goal of the assignment is to use the Python programming skils gained in the course to write a program that can analyse and visualize a dataset.  
  
**Project Motivation**
  
For my project, I have chosen to implement a Latent Semantic Analysis [1] on a corpus of English language documents with the aim of providing synonymous terms. 
My rationale is that the LSA approach appears very generic to me and can be also applied in the business context whenever other feature vectors are available (e.g. a vector of attributes per customer instead of a vector of words), and the resulting matrices can be applied flexibly to answer a number of questions on feature or document (or subject) similarity, context or importance.
  
**The dataset**
  
The dataset I have used is a moderately sized corpus of 2225 documents from the BBC news website (in English) corresponding to stories in five topical areas from 2004-2005: business, entertainment, politics, sport, and technology [2].
  

**Tools used**
  
In order for this notebook to run as intended, you should have the following Python modules installed:  
  
* `os`, `urllib2`, `zipfile` - for reading the corpus
* `re`, `string` - for string processing
* `numpy`,`pandas`, `sklearn` - for processing the data and running the LSA
* `bokeh`, `IPython.html`, `ipywidgets` - for the interactive visualization of the results 
  
**Implementation overview**
  
Here is an overview of the data processing flow and the components:

<img src="app_struct.png">

## 1. Offline part

This is the main part of the program. The offline part runs only once, and parses the corpus, runs the latent semantic analysis functions over it and produces the necessary outputs for the interactive search of similar terms.

### 1.1. Read and clean the corpus

First we load the required modules and fetch the zipped corpus data [3]:

In [1]:
import numpy as np
import pandas as pd
import os
import urllib2

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer,HashingVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

In [2]:
# Check if corpus file exists and if not - download it

if not os.path.isfile('corpus.zip'):
    print 'Downloading data from GitHub...'
    
    # Download the zip into the data/ directory
    response = urllib2.urlopen('https://github.com/datafeelings/data_602_final_project/raw/master/data/bbc-fulltext.zip')
    down_file= response.read()

    with open("corpus.zip", 'w') as f:
        f.write(down_file)
    print 'Saved "corpus.zip" in the notebook directory'

In [3]:
# Read files from zip
arc_path = "corpus.zip"

def read_corpus_from_zip(archive_path):
    ''' Reads a corpus of .txt documents stored in subfolders within a .zip archive
    
    Args:
        archive_path: path to the zipped corpus of .txt documents

    Returns:
        Dict with documents and terms
    '''
    
    import zipfile
    import os
    
    with zipfile.ZipFile(archive_path) as z:

        docs = []
        terms = []
        # need to read only .txt files but not those hidden ones created by the MacOS

        for filename in [f for f in z.namelist() if (not f.startswith('_'))& f.endswith('.txt')]:
            with z.open(filename) as f:
                docs.append(filename)
                words = f.read().replace('\n', '')
                terms.append(words)

    print 'Created a corpus with {} documents'.format(len(docs))
    return {'docs' : docs,'terms' : terms}

corpus = read_corpus_from_zip(arc_path)

Created a corpus with 2225 documents


Clean and stem the strings before passing to the tokenizer that will count term occurrences per document [3]:

In [4]:
# Process terms: Remove punctuation & numbers, lowercase, remove articles and short words

def clean_strings(string_):
    '''Removes punctuation & numbers, turns to lowercase, removes 1- or 2-letter terms from a string of text
    
    Args:
        string_: (str) path to the zipped corpus of .txt documents

    Returns:
        (str) processed string
    '''
    
    import string
    import re
    
    string_ = string_.lower()  # lowercase
    string_ = re.sub(r'[^\x00-\x7f]',r' ',string_) # remove non-unicode characters
    string_ = re.sub(r'[\d]',r'',string_) # retain only letters
    string_ = re.sub(r'[%$()!.,]',r' ',string_) # retain only letters
    string_ = re.sub(r'[ +]',r' ',string_) # collapse multiple whitespaces
    string_ = re.sub(r' *\b[a-zA-Z]{1,2}\b',r'',string_) # remove all 1- or 2-word terms
    
    return string_


corpus['terms'] = [clean_strings(item) for item in corpus['terms']]


Print the first 300 symbols of the first document after parsing:

In [5]:
# Print the first 300 symbols of the first document
corpus['terms'][0][:300]

' sales boost time warner profitquarterly profits media giant timewarner jumped  for the three months december  from year-earlier the firm  which now one the biggest investors google  benefited from sales high-speed internet connections and higher advert sales  timewarner said fourth quarter sales ro'

### 1.2. Run the LSA 

This part relies extensively on the LSA implementation in the Scikit-Learn module. The benefits of this approach vs. custom-implemented functions is the speed of processing. 

The main reference for this part is the following tutorial by Peter Prettenhofer and Lars Buitinck [5]:  
  
http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py  
  
Other references covering the theory of the LSA and topic modeling are [6,7,8].

First, a `TfidfVectorizer` constructs a term-document matrix from the cleaned corpus. In the cells of this matrix are the TF-IDF weights of each term for each document.

In [6]:
t0 = time()

vectorizer = TfidfVectorizer(max_df=0.7,min_df=2, stop_words='english',use_idf=True)
# max_df=0.7 means that the terms appearing than more in the 70% of the documents are ignored
# min_df=2 means that the terms appearing in less than 2 documents are ignored

X = vectorizer.fit_transform(corpus['terms'])

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)

done in 0.789057s
n_samples: 2225, n_features: 16723


Next, the actual Singular Value Decomposition of the term-document matrix is calculated:
  
* The TDM is decomposed into the matrix X of the terms mapped to "concepts"(or semantic dimensions, or components), the square matrix S of singular values with "concepts" as rows/columns, the matrix Y of the "concepts" mapped to documents,  
* Using SVD truncation (rank-lowering), the dimension of the matrices are reduced so that S is a 100x100 matrix of the top 100 semantic dimensions (the nuber 100 selected based on literature review)

In [7]:
t0 = time()

svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
# n_components = 100 means that the matrix is reduced to 100 components (top 100 singular values)
# n_iter = 7 - Number of iterations for randomized SVD solver. 

normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

# use transpose to generate a matrix for terms(features) instead of documents
X = lsa.fit_transform(X.transpose()) 

print("done in %fs" % (time() - t0))
print("n_terms: %d, n_components: %d" % X.shape)

done in 1.310288s
n_terms: 16723, n_components: 100


In [8]:
X

array([[ 0.11322606, -0.08858838, -0.11481925, ...,  0.02483692,
         0.0308393 ,  0.12903484],
       [ 0.11673417, -0.10178572, -0.12229693, ..., -0.1090511 ,
         0.06981183, -0.03725003],
       [ 0.11532662, -0.10773052, -0.02530448, ...,  0.0298613 ,
        -0.01788788,  0.03954373],
       ..., 
       [ 0.24039674, -0.20552223, -0.23199312, ...,  0.01500166,
         0.05132643, -0.03396719],
       [ 0.13739699, -0.15670377, -0.19861743, ...,  0.01157532,
         0.18498143, -0.20329173],
       [ 0.09546642, -0.124781  , -0.11831286, ..., -0.00733596,
        -0.03006833, -0.11851548]])

In [9]:
X.shape

(16723, 100)

Now we have a matrix **X** with M = 16.723 terms described by N = 100 components representing the "semantic dimensions".  
We can:
  
1) Look at the terms that have the highest scores within each dimension  
2) Calculate pairwise similarity between two term vectors (1xN each) using cosine distance 
  
So for any term we can provide a set of e.g. top 10 synonyms using the top values from the similarity comparison.

### 1.3. Extract the top terms per dimension

In this part we'll provide some insight into what topics the 100 semantic dimensions inferred from the corpus describe.  
First we fetch the names of the terms:

In [10]:
# The array of term names
terms = vectorizer.get_feature_names()
terms[:10]

[u'aaa',
 u'aaas',
 u'aac',
 u'aaron',
 u'abacus',
 u'abandon',
 u'abandoned',
 u'abandoning',
 u'abandonment',
 u'abating']

Now find top terms per semantic dimension

In [11]:
# semantic dimensions = components, are the columns, one row per each term
# Convert to a Pandas df for easier operations
terms_comps = pd.DataFrame(X) 
terms_comps.head(2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.113226,-0.088588,-0.114819,-0.066678,-0.017645,0.028401,-0.020780,-0.293838,-0.176486,0.032807,...,0.020546,-0.064412,0.018931,0.109256,-0.063180,0.108095,0.077723,0.024837,0.030839,0.129035
1,0.116734,-0.101786,-0.122297,-0.047617,-0.036581,0.017029,-0.023478,-0.291808,-0.139500,0.071898,...,0.013701,0.239071,-0.020271,-0.023135,-0.017255,-0.037946,-0.050270,-0.109051,0.069812,-0.037250


Now we can extract top-scoring terms from each dimension to get a feeling of what topics they describe

In [12]:
# Function to filter top n words per semantic dimension

def get_top_x_term_df(inp_matrix, term_names, top_x):
    ''' Returns a dataframe with top-scoring terms per semantic dimension
    
    Args:
        inp_matrix: (DataFrame) Matrix mapping terms in rows to weights per semantic component in columns
        term_names: (List) list with the vocabulary of the terms
        top_x: (int) number of top-scoring terms per dimension to return

    Returns:
        (DataFrame) dataframe holding the term, the dimension and its score
    '''
        
    # Convert into long pandas table and use built-in top-n function, then use the index of the top scoring 
    # terms to filter the melted term_component df
    tc_matrix = inp_matrix.copy()
    val_cols = tc_matrix.columns
    tc_matrix['term'] = term_names
    tc_melt = pd.melt(tc_matrix, id_vars=['term'], value_vars=list(val_cols),var_name='sem_dim', value_name='weight')
    top_weights = tc_melt.groupby(['sem_dim'])['weight'].nlargest(top_x)
    top_df = tc_melt.ix[top_weights.index.get_level_values(1),:]
    return top_df


In [20]:
top_terms = get_top_x_term_df(terms_comps,terms, 10) # extract top 10 words per dimension
sem_dim_descr = top_terms.groupby('sem_dim')['term'].apply(lambda x: "%s" % ', '.join(x)) # reference [9]
sem_dim_descr.head(10)

sem_dim
0    far, added, make, time, way, long, end, major,...
1    labour, election, manifesto, conservative, pol...
2    analyst, analysts, growing, market, according,...
3    film, cinematography, nominated, actress, acto...
4    uses, lets, streaming, downloadable, use, user...
5    litigation, maintains, damages, jurisdiction, ...
6    meaningless, embattled, yugansk, rosneft, pump...
7    lock, scrum, forwards, grewcock, kicker, engla...
8    league, club, defender, clubs, football, premi...
9    russians, tool, windows, program, log, tools, ...
Name: term, dtype: object

We can see how the first dimensions show terms related to the same topic, e.g. dimension 1 appears to be related to politics, dimension 2 to "stock markets or business", dimension 3 is about films and actors, 4 about internet usage, 5 about court and litigation, however, the dimension 0 is unclear when judged alone.

### 1.4. Calculate similarity between terms

Now we can finally use the matrix of the terms mapped to the semantic dimensions to calculate the distance between a given term and each other term - and then pick the closest ones. Here, the Pandas and list documentation was used as reference [10,11].

In [14]:
# Calculate cosine distances from 1 vector to all the other vectors to find close terms

def top_synonyms(tc_matrix,vocab,search_term,search_n):
    ''' For a given term-component matrix and a search term, returns top n most similar terms
    
    Args:
        tc_matrix: (DataFrame) Matrix mapping terms in rows to weights per semantic component in columns
        vocab: list with all the terms in the vocabulary; the length must match tc_matrix row number
        search_term: (str) string with search term
        search_n: (int) number of most similar terms to return

    Returns:
        (List) List of tuples with (term,similarity) to the search term
        
    '''
    
    # Preprocess user input
    search_term = clean_strings(search_term).split(' ')[0] # preprocess the input and take only the first string
    
    try: 
        search_index = vocab.index(search_term)
        term_vec = tc_matrix[search_index,:].reshape(1, -1) # vector for the search item
    
        # find distances to each other term, as the values are normalized can use linear kernel instead of cosine
        term_dist = linear_kernel(term_vec, tc_matrix) 

        vals_max = np.sort(term_dist).take(range(-(search_n+1),0)) # top n+1 closest terms (as the term itself will be closest)
        vals_max = vals_max[::-1] 
        terms_idx_max = [np.where(term_dist[0] == vals_max[idx])[0][0] for idx in range(0,vals_max.shape[0])] 
        terms_max = [terms[i] for i in terms_idx_max] 
        
        output = zip(terms_max, vals_max)[1:] # exclude the matched term itelf from the output
        
        # remove duplicates and sort again
        output = set(output) 
        output = sorted(output, key=lambda tup: tup[1])
        
        return output
    
    except:
        print("Term not found in the vocabulary")
        return ('',0.0)
    
    


The function `top_synonyms` returns the similar terms using the matrix `X` and the vocabulary list `terms` defined above:

In [15]:
syn = top_synonyms(X, terms, 'Car', 10)
syn

[(u'suvs', 0.84374154366331178),
 (u'saab', 0.84552522738624325),
 (u'vehicles', 0.84767965294607106),
 (u'carmaker', 0.84870587296567201),
 (u'lexus', 0.8531752060932396),
 (u'diesel', 0.87026015727237005),
 (u'marques', 0.89193622549154128),
 (u'cars', 0.9229350141782231),
 (u'marque', 0.93718056654147608),
 (u'motor', 0.95454353841950124)]

## 2. Online part

This part is the interactive component that uses the functions and input variables defined in the first part to interactively retrieve similar terms based on user input [12,13].

### 2.1. Interactive chart with similar terms
  
First we define a plotting function:

In [16]:
# Load graphics

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

Loading BokehJS ...

In [17]:
# Draw a bar chart based on the input

def top_syn_bar(search_term,search_n, min_score):
    '''
    WRITE DOCSTRING
    '''
    from bokeh.palettes import PuRd
    from bokeh.charts.attributes import cat, color
    from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, NumeralTickFormatter
    
    syns = top_synonyms(X, terms, search_term,search_n) # get the data
    
    # Set up the data for the Bokeh bar chart
    chart_data = pd.DataFrame(syns)
    chart_data.columns = ["term","score"]
    chart_data = chart_data.sort_values('score') # sort desc order
    chart_data['x_lp'] = 0 # add a column for x annotation position
    chart_data = chart_data[chart_data['score'] > min_score] # remove non-relevant records
     
    y = list(chart_data['term'])
    x = list(chart_data['score'])
    
    f_title = 'Top Related Terms for: '+ search_term
    
    p = figure(title=f_title,y_range=y)

    p.hbar(y=y, height=0.5, left=0,right=x, 
           color="navy")

    p.xaxis.axis_label = "Relative Term Similarity"
    p.xaxis[0].formatter = NumeralTickFormatter(format="0%")
    p.yaxis.visible = False

    
    labels = LabelSet(x='x_lp', y='term', text='term', level='glyph',text_color="white",
              x_offset=5, y_offset=-8, source=ColumnDataSource(chart_data), 
                      render_mode='canvas')
    
    p.add_layout(labels)
    show(p)

**Interactive output**
  
And then we define the interactive modification of the plotting function using input widgets.
Below this block you should see three widgets and a horizontal bar chart displaying the similarity of the found terms to the provided input. The widgets allow you to modify:
  
1) The input term  
2) The maximum number of the related terms that is returned  
3) The minimum level of similarity (0-100%) that is used to filter the returned list  

In [18]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from ipywidgets import HBox, Label, IntSlider

def f(term, n_terms,min_score):
    '''Provides interactive output of top_syn_bar() modifiable by widgets'''
    try:
        top_syn_bar(term,n_terms,min_score)
    except:
        print 'Term not found: Chart not available'


# 'interact' allows the widgets to modify the output of the function defined as the first argument

interact(f, term = widgets.Text(value='car',description='Search Term: '),
         n_terms=widgets.IntSlider(min=-10,max=25,step=1,value=10,description='Number of related terms: '),
         min_score=widgets.FloatSlider(min=0,max=1,step=0.05,value=0.6, readout_format='.0%',
                                       description='Minimum similarity: '))

### 2.2.  Discussion

By trying out a few keywords it becomes apparent that the performance of the similarity search depends strongly on the topical coverage of the corpus. As we know, the BBC article corpus used here covers 5 topical areas: business, entertainment, politics, sport, and technology. Keywords from these areas such as "money", "car", "Nirvana", "Chelsea" (for the soccer team) or "console" subjectively seem to perform better (relevant terms are provided) than the terms outside of the topical scope of the corpus (e.g. "pasta", "tomato" - are not in the filtered vocabulary, and "heat" only provides "synonyms" related to overheating in supercomputers). 
  
Therefore, this analysis could be repeated on a larger corpus in the future to get more precise and relevant results.

---
  
## Reference

  
0. [Wikipedia: Latent Semantic Analysis](https://en.wikipedia.org/wiki/Latent_semantic_analysis)  
  
1. [D. Greene and P. Cunningham. "Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering", Proc. ICML 2006.](http://mlg.ucd.ie/datasets/bbc.html)
  
2. [Stackoverflow: How to download a ZIP file from a site Python](https://stackoverflow.com/questions/16760992/how-to-download-a-zip-file-from-a-site-python)  

3. [Python: 7.2. re — Regular expression operations](https://docs.python.org/2/library/re.html)  

4. [Clustering text documents using k-means](http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py)  

5. [Cambridge University Press: Scoring, term weighting and the vector space model](https://nlp.stanford.edu/IR-book/html/htmledition/scoring-term-weighting-and-the-vector-space-model-1.html)

6. [Mat Kelcey: latent semantic analysis via the singular value decomposition (for dummies)](http://matpalm.com/lsa_via_svd/)

7. [William Webber: Lecture 7: Matrix decomposition and LSA](http://www.williamwebber.com/research/teaching/comp90042/2014s1/lect/l07.pdf)
  
8. [Stackoverflow: Pandas groupby: How to get a union of strings](https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings)
  
9. [pandas: powerful Python data analysis toolkit](http://pandas.pydata.org/pandas-docs/version/0.17.0/index.html)
  
10. [Stackoverflow: How to sort list/tuple of lists/tuples](https://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples)
   
11. [Bokeh: User Guide](http://bokeh.pydata.org/en/latest/docs/user_guide.html#userguide)
  
12. [IPython: Using Interact](http://nbviewer.jupyter.org/github/quantopian/ipython/blob/master/examples/Interactive%20Widgets/Using%20Interact.ipynb)

